In [209]:
import numpy as np
import pandas as pd
import yfinance as yf
from tools import sma

# Turnaround Tuesday - Strategie im NASDAQ 100

Zunächst werden die Kursdaten für den NASDAQ 100 geladen und bereinigt

In [210]:
SYMBOL = "^NDX"
SYMBOL = "QQQ"


stock = yf.download(SYMBOL)

stock = stock[~(stock.High == stock.Low) & ~(stock.Open == stock.Close)]
stock = stock.dropna()

df = stock


[*********************100%%**********************]  1 of 1 completed


Ein paar Indikatoren für spätere Auswertungen

## Jetzt zur Strategie
...

In [211]:
from backtesting import Strategy


class turnaround_day(Strategy):
    """
    strategy for trend_trading
    """
    
    sma_interval = 5
    turnaround_day = 0 # Monday

    def ohlc(self, pos=-1) -> tuple:
        """
        helper function for ohlc data
        """
        return (
            self.data.Open[pos],
            self.data.High[pos],
            self.data.Low[pos],
            self.data.Close[pos],
        )


    def init(self):
        super().init()

        self.sma = self.I(sma, self.data.Close.s, self.sma_interval)


    def next(self):
        # super().next()

        _open, _high, _low, _close = self.ohlc()

        # trade management for an existing trade
        for trade in self.trades:
            if trade.is_long:
                if len(self.data) - self.trades[-1].entry_bar >1:

                    if self.data.Close[-1] > self.data.Close[-2]:
                        trade.close()

        if self.data.df.iloc[-1].name.dayofweek == self.turnaround_day: # Monday
            if self.data.Close[-2] > self.data.Close[-1] and self.data.Close[-3] > self.data.Close[-2]:
                if _close > self.sma[-1] : 
                    self.buy()

In [212]:
from backtesting import Backtest

bt = Backtest(
    df["2000-01-01":], turnaround_day, cash=100_000, commission=0.002, trade_on_close=False
)

stats = bt.run()
bt.plot(superimpose=False, open_browser=False)
stats

Start                     2000-01-03 00:00:00
End                       2024-05-24 00:00:00
Duration                   8908 days 00:00:00
Exposure Time [%]                      1.0841
Equity Final [$]                 94961.391045
Equity Peak [$]                 105378.585768
Return [%]                          -5.038609
Buy & Hold Return [%]              383.324551
Return (Ann.) [%]                   -0.213772
Volatility (Ann.) [%]                2.622298
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -12.915526
Avg. Drawdown [%]                  -12.915526
Max. Drawdown Duration     8347 days 00:00:00
Avg. Drawdown Duration     8347 days 00:00:00
# Trades                                   17
Win Rate [%]                        52.941176
Best Trade [%]                       2.351583
Worst Trade [%]                     -7.138017
Avg. Trade [%]                    

In [213]:
stats, heatmap = bt.optimize(
    turnaround_day=range(0, 5,1),
    sma_interval=range(20, 50,5),
    maximize="Equity Final [$]",#"Profit Factor",  
    max_tries=500,
    random_state=0,
    return_heatmap=True,
)

stats = bt.run(**stats._strategy._params)
bt.plot(superimpose=False, open_browser=True)
stats


/Users/produktmanagement/Python/github/backtesting-trading-strategies/.venv/lib/python3.12/site-packages/backtesting/backtesting.py:1375: UserWarning: For multiprocessing support in `Backtest.optimize()` set multiprocessing start method to 'fork'.
  warnings.warn("For multiprocessing support in `Backtest.optimize()` "


Start                     2000-01-03 00:00:00
End                       2024-05-24 00:00:00
Duration                   8908 days 00:00:00
Exposure Time [%]                    6.833114
Equity Final [$]                169953.345591
Equity Peak [$]                 172752.963398
Return [%]                          69.953346
Buy & Hold Return [%]              383.324551
Return (Ann.) [%]                    2.219562
Volatility (Ann.) [%]                5.155322
Sharpe Ratio                         0.430538
Sortino Ratio                        0.718313
Calmar Ratio                         0.120558
Max. Drawdown [%]                  -18.410777
Avg. Drawdown [%]                   -2.211133
Max. Drawdown Duration     2892 days 00:00:00
Avg. Drawdown Duration      259 days 00:00:00
# Trades                                  113
Win Rate [%]                        69.911504
Best Trade [%]                       8.054879
Worst Trade [%]                     -6.892339
Avg. Trade [%]                    

In [214]:
heatmap.sort_values().iloc[-10:]

turnaround_day  sma_interval
0               40              124477.879467
1               20              124481.932225
0               30              125279.234671
1               25              125453.641953
0               45              129068.855887
                35              129752.724963
1               30              142088.469103
                35              144028.581421
                40              163375.011590
                45              169953.345591
Name: Equity Final [$], dtype: float64

In [215]:
trades = stats["_trades"]

jetzt wird überprüft welche Tage für die Ein- und Ausstiege besser sein könnten.

The Winner is ...

In [216]:
stats["_strategy"]

<Strategy turnaround_day(turnaround_day=1,sma_interval=45)>

Die Win Rate beim Trading bezieht sich auf das Verhältnis der Anzahl der gewonnenen Trades zur Gesamtanzahl der Trades. 
Wenn jemand beispielsweise 70% seiner Trades gewinnt, beträgt seine Win Rate 70%.

In [217]:
stats["Win Rate [%]"]

69.91150442477876

Durchschnittlicher Gewinn in Prozent. 
Wie viel Gewinn oder Verlust wurden im Durchschnitt pro Trade erzielt.

In [218]:
stats["Avg. Trade [%]"]

0.4706553160297222

Der Profit-Faktor stellt die Verhältnisse zwischen Gewinnen und Verlusten dar.
Der Profit-Faktor wird berechnet, indem die Summe aller Profite (Gewinne) durch die Summe aller Verluste (einschließlich Gebühren und Kommissionen) für den gesamten Handelszeitraum geteilt wird.
Ein Profit-Faktor größer als 1 zeigt an, dass die Strategie profitabel ist. Je höher der Wert, desto besser.

Ein guter Profit-Faktor liegt normalerweise zwischen 1,75 und 41.
Ein Wert über 1,75 zeigt eine solide Strategie an, während Werte über 4 außergewöhnlich gut sind.

In [219]:
stats["Profit Factor"]

2.041052872991066

In [220]:
from backtesting import Strategy

class turnaround_day_optimized(Strategy):
    """
    strategy for trend_trading
    """
    
    sma_interval = 40

    def ohlc(self, pos=-1) -> tuple:
        """
        helper function for ohlc data
        """
        return (
            self.data.Open[pos],
            self.data.High[pos],
            self.data.Low[pos],
            self.data.Close[pos],
        )


    def init(self):
        super().init()

        self.sma = self.I(sma, self.data.Close.s, self.sma_interval)


    def next(self):
        # super().next()

        _open, _high, _low, _close = self.ohlc()

        # trade management for an existing trade
        for trade in self.trades:
            if trade.is_long:
                if len(self.data) - self.trades[-1].entry_bar >1:

                    if self.data.Close[-1] > self.data.Close[-2]:
                        trade.close()

        if self.data.df.iloc[-1].name.dayofweek == 0 or self.data.df.iloc[-1].name.dayofweek == 1:
            if self.data.Close[-2] > self.data.Close[-1] and self.data.Close[-3] > self.data.Close[-2]:
                #if _close > self.sma[-1] : 
                self.buy()

In [221]:
from backtesting import Backtest

bt = Backtest(
    df["2000-01-01":], turnaround_day_optimized, cash=100_000, commission=0.002, trade_on_close=False
)

stats = bt.run()
bt.plot(superimpose=False, open_browser=True)
stats

Start                     2000-01-03 00:00:00
End                       2024-05-24 00:00:00
Duration                   8908 days 00:00:00
Exposure Time [%]                   23.111038
Equity Final [$]                291532.819692
Equity Peak [$]                 309794.438997
Return [%]                          191.53282
Buy & Hold Return [%]              383.324551
Return (Ann.) [%]                    4.528511
Volatility (Ann.) [%]               15.301742
Sharpe Ratio                         0.295947
Sortino Ratio                        0.484224
Calmar Ratio                         0.207318
Max. Drawdown [%]                  -21.843286
Avg. Drawdown [%]                    -4.47789
Max. Drawdown Duration     2080 days 00:00:00
Avg. Drawdown Duration      135 days 00:00:00
# Trades                                  370
Win Rate [%]                        61.621622
Best Trade [%]                        12.4002
Worst Trade [%]                    -13.578445
Avg. Trade [%]                    

In [222]:
trades

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,937,50,52,106.713000,108.000000,1205.919000,0.012060,2000-03-15,2000-03-17,2 days
1,2300,326,329,43.987802,46.200001,5088.058238,0.050291,2001-04-25,2001-05-01,6 days
2,2335,349,351,45.520860,45.540001,44.693324,0.000420,2001-05-30,2001-06-01,2 days
3,3117,453,455,34.108081,35.270000,3621.703207,0.034066,2001-10-31,2001-11-02,2 days
4,2933,542,545,37.484820,37.630001,425.816521,0.003873,2002-03-13,2002-03-18,5 days
...,...,...,...,...,...,...,...,...,...,...
108,378,6012,6014,431.270824,433.940002,1008.949576,0.006189,2024-02-07,2024-02-09,2 days
109,380,6017,6019,432.122530,434.890015,1051.644248,0.006404,2024-02-14,2024-02-16,2 days
110,388,6021,6023,425.399088,439.649994,5529.351578,0.033500,2024-02-21,2024-02-23,2 days
111,387,6031,6033,441.200647,445.809998,1783.818535,0.010447,2024-03-06,2024-03-08,2 days
